In [1]:
import os
import duckdb
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
comet_ml is installed but `COMET_API_KEY` is not set.


In [2]:
db_path = '../duck_db/isrecon_AIS11.duckdb'

with duckdb.connect(database=db_path, read_only=True) as conn:
    query = 'SELECT article_id, title, abstract FROM papers LIMIT 500'
    df = conn.execute(query).fetchdf()

In [3]:
db_path = '../duck_db/isrecon_AIS11.duckdb'

In [4]:
with duckdb.connect(database=db_path, read_only=True) as conn:
    query = '''SELECT article_id,authors, year, title, journal, abstract, keywords, citation_count FROM papers
    WHERE title IN (
    'Technology Adaptation: The Case of a Computer-Supported Inter-organizational Virtual Team',
    'How Do Suppliers Benefit from Information Technology Use in Supply Chain Relationships?',
    'A Multilevel Model of Resistance to Information Technology Implementation',
    'Understanding User Responses to Information Technology: A Coping Model of User Adaptation',
    'A Comprehensive Conceputalization of the Post-Adoptive Behaviors Associated with IT-Enabled Work Systems',
    'Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis',
    'Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective',
    'How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance',
    'Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation',
    'The Integrative Framework of Technology Use: An Extension and Test',
    'Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use',
    'An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups',
    'Capturing Bottom-Up Information Technology Use Processes: A Complex Adaptive Systems Model',
    'Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers',
    'Interfirm IT Capability Profiles and Communications for Cocreating Relational Value: Evidence from the Logistics Industry',
    'A Dramaturgical Model of the Production of Performance Data',
    'The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption',
    'When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances',
    'Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance',
    'An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance',
    'Toward Generalizable Sociomaterial Inquiry: A Computational Approach for Zooming In and Out of Sociomaterial Routines',
    'Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns',
    'Information Technology Use as a Learning Mechanism: The Impact of IT Use on Knowledge Transfer Effectiveness, Absorptive Capacity, and Franchisee Performance',
    'ICT, Intermediaries, and the Transformation of Gendered Power Structures',
    'Multiplex Appropriation in Complex Systems Implementation: The Case of Brazil''s Correspondent Banking System',
    'Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations',
    'Capturing the Complexity of Malleable IT Use: Adaptive Structuration Theory for Individuals',
    'A Temporally Situated Self-Agency Theory of Information Technology Reinvention'
);'''
    df_article = conn.execute(query).fetchdf()

In [5]:
print(df_article)

    article_id                                            authors  year  \
0          926               Bartelt, Valerie L.; Dennis, Alan R.  2014   
1         1033                 Beaudry, Anne; Pinsonneault, Alain  2005   
2         1658         Burton-Jones, Andrew; Gallivan, Michael J.  2007   
3         6541                                       Kim, Sung S.  2009   
4         7061                  Lapointe, Liette; Rivard, Suzanne  2005   
5         7383                                  Leonardi, Paul M.  2013   
6         7714  Limayem, Moez; Hirt, Sabine Gabriele; Cheung, ...  2007   
7         9183      Nevo, Saggi; Nevo, Dorit; Pinsonneault, Alain  2016   
8         9462             Ortiz de Guinea, Ana; Markus, M. Lynne  2009   
9         9464                Ortiz de Guinea, Ana; Webster, Jane  2013   
10        9957                Polites, Greta L.; Karahanna, Elena  2013   
11       10360    Ray, Gautam; Muhanna, Waleed A.; Barney, Jay B.  2005   
12       10863           

In [7]:
with duckdb.connect(database=db_path, read_only=True) as conn:
    query = '''SELECT sentences.article_id, para_id, last_section_title  FROM sentences
                JOIN papers ON sentences.article_id = papers.article_id
                WHERE title IN (
                'Technology Adaptation: The Case of a Computer-Supported Inter-organizational Virtual Team',
                'How Do Suppliers Benefit from Information Technology Use in Supply Chain Relationships?',
                'A Multilevel Model of Resistance to Information Technology Implementation',
                'Understanding User Responses to Information Technology: A Coping Model of User Adaptation',
                'A Comprehensive Conceputalization of the Post-Adoptive Behaviors Associated with IT-Enabled Work Systems',
                'Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis',
                'Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective',
                'How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance',
                'Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation',
                'The Integrative Framework of Technology Use: An Extension and Test',
                'Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use',
                'An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups',
                'Capturing Bottom-Up Information Technology Use Processes: A Complex Adaptive Systems Model',
                'Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers',
                'Interfirm IT Capability Profiles and Communications for Cocreating Relational Value: Evidence from the Logistics Industry',
                'A Dramaturgical Model of the Production of Performance Data',
                'The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption',
                'When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances',
                'Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance',
                'An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance',
                'Toward Generalizable Sociomaterial Inquiry: A Computational Approach for Zooming In and Out of Sociomaterial Routines',
                'Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns',
                'Information Technology Use as a Learning Mechanism: The Impact of IT Use on Knowledge Transfer Effectiveness, Absorptive Capacity, and Franchisee Performance',
                'ICT, Intermediaries, and the Transformation of Gendered Power Structures',
                'Multiplex Appropriation in Complex Systems Implementation: The Case of Brazil''s Correspondent Banking System',
                'Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations',
                'Capturing the Complexity of Malleable IT Use: Adaptive Structuration Theory for Individuals',
                'A Temporally Situated Self-Agency Theory of Information Technology Reinvention'
                );'''
    df_sentences= conn.execute(query).fetchdf()

In [8]:
print(df_sentences)

       article_id    para_id last_section_title
0             926      926_0               None
1             926      926_1           Abstract
2             926      926_1           Abstract
3             926      926_1           Abstract
4             926      926_1           Abstract
...           ...        ...                ...
17383       12686  12686_233   About the Author
17384       12686  12686_234   About the Author
17385       12686  12686_234   About the Author
17386       12686  12686_234   About the Author
17387       12686  12686_234   About the Author

[17388 rows x 3 columns]


In [9]:
df_sentences = df_sentences.drop_duplicates(subset=['para_id'])
print(df_sentences.head())

    article_id para_id last_section_title
0          926   926_0               None
1          926   926_1           Abstract
10         926   926_2     Introduction 1
11         926   926_3     Introduction 1
14         926   926_4     Introduction 1


In [10]:
with duckdb.connect(database=db_path, read_only=True) as conn:
    query = '''SELECT para_id, paragraph FROM paragraphs
                JOIN papers ON paragraphs.article_id = papers.article_id
                WHERE title IN (
                'Technology Adaptation: The Case of a Computer-Supported Inter-organizational Virtual Team',
                'How Do Suppliers Benefit from Information Technology Use in Supply Chain Relationships?',
                'A Multilevel Model of Resistance to Information Technology Implementation',
                'Understanding User Responses to Information Technology: A Coping Model of User Adaptation',
                'A Comprehensive Conceputalization of the Post-Adoptive Behaviors Associated with IT-Enabled Work Systems',
                'Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis',
                'Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective',
                'How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance',
                'Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation',
                'The Integrative Framework of Technology Use: An Extension and Test',
                'Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use',
                'An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups',
                'Capturing Bottom-Up Information Technology Use Processes: A Complex Adaptive Systems Model',
                'Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers',
                'Interfirm IT Capability Profiles and Communications for Cocreating Relational Value: Evidence from the Logistics Industry',
                'A Dramaturgical Model of the Production of Performance Data',
                'The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption',
                'When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances',
                'Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance',
                'An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance',
                'Toward Generalizable Sociomaterial Inquiry: A Computational Approach for Zooming In and Out of Sociomaterial Routines',
                'Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns',
                'Information Technology Use as a Learning Mechanism: The Impact of IT Use on Knowledge Transfer Effectiveness, Absorptive Capacity, and Franchisee Performance',
                'ICT, Intermediaries, and the Transformation of Gendered Power Structures',
                'Multiplex Appropriation in Complex Systems Implementation: The Case of Brazil''s Correspondent Banking System',
                'Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations',
                'Capturing the Complexity of Malleable IT Use: Adaptive Structuration Theory for Individuals',
                'A Temporally Situated Self-Agency Theory of Information Technology Reinvention'
                );'''
    df_paragraphs= conn.execute(query).fetchdf()

In [11]:
print(df_paragraphs)

       para_id                                          paragraph
0       9957_0  THE EMBEDDEDNESS OF INFORMATION SYSTEMS HABITS...
1       9957_1  Despite recent interest in studying informatio...
2       9957_3  The psychological construct of habit has attra...
3       9957_4  Further, within organizations, IS use is almos...
4       9957_5  The objective of the current paper is to contr...
...        ...                                                ...
2484  1658_152  6. Scholars could begin multilevel theory deve...
2485  1658_153  7. Because a number of different structures ca...
2486  1658_154  8. Because similar structures can result in di...
2487  1658_155  10. When operationalizing collective construct...
2488  1658_156  11. Researchers should be clear in how they op...

[2489 rows x 2 columns]


In [12]:
merged_df = df_sentences.merge(df_paragraphs, on=['para_id'])

In [13]:
print(merged_df)

      article_id    para_id  \
0            926      926_0   
1            926      926_1   
2            926      926_3   
3            926      926_4   
4            926      926_5   
...          ...        ...   
2484       12686  12686_211   
2485       12686  12686_212   
2486       12686  12686_213   
2487       12686  12686_215   
2488       12686  12686_216   

                                     last_section_title  \
0                                                  None   
1                                              Abstract   
2                                        Introduction 1   
3                                        Introduction 1   
4                                        Introduction 1   
...                                                 ...   
2484                                         Conclusion   
2485                                         Conclusion   
2486                                         Conclusion   
2487  Despite its limitations and ope

In [14]:
with duckdb.connect(database=db_path, read_only=True) as conn:
    query = '''SELECT para_id,label, ent_id, level_3, FROM entities
                JOIN papers ON entities.article_id = papers.article_id
                
                WHERE title IN (
                'Technology Adaptation: The Case of a Computer-Supported Inter-organizational Virtual Team',
                'How Do Suppliers Benefit from Information Technology Use in Supply Chain Relationships?',
                'A Multilevel Model of Resistance to Information Technology Implementation',
                'Understanding User Responses to Information Technology: A Coping Model of User Adaptation',
                'A Comprehensive Conceputalization of the Post-Adoptive Behaviors Associated with IT-Enabled Work Systems',
                'Information Technology and the Performance of the Customer Service Process: A Resource-Based Analysis',
                'Toward a Deeper Understanding of System Usage in Organizations: A Multilevel Perspective',
                'How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance',
                'Predicting Different Conceptualizations of System Use: The Competing Roles of Behavioral Intention, Facilitating Conditions, and Behavioral Expectation',
                'The Integrative Framework of Technology Use: An Extension and Test',
                'Why Break the Habit of a Lifetime? Rethinking the Roles of Intention, Habit, and Emotion in Continuing Information Technology Use',
                'An Alternative to Methodological Individualism: A Non-Reductionist Approach to Studying Technology Adoption by Groups',
                'Capturing Bottom-Up Information Technology Use Processes: A Complex Adaptive Systems Model',
                'Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers',
                'Interfirm IT Capability Profiles and Communications for Cocreating Relational Value: Evidence from the Logistics Industry',
                'A Dramaturgical Model of the Production of Performance Data',
                'The Embeddedness of Information Systems Habits in Organizational and Individual Level Routines: Development and Disruption',
                'When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances',
                'Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance',
                'An Investigation of Information Systems Use Patterns: Technological Events as Triggers, the Effect of Time, and Consequences for Performance',
                'Toward Generalizable Sociomaterial Inquiry: A Computational Approach for Zooming In and Out of Sociomaterial Routines',
                'Coping with Information Technology: Mixed Emotions, Vacillation, and Nonconforming Use Patterns',
                'Information Technology Use as a Learning Mechanism: The Impact of IT Use on Knowledge Transfer Effectiveness, Absorptive Capacity, and Franchisee Performance',
                'ICT, Intermediaries, and the Transformation of Gendered Power Structures',
                'Multiplex Appropriation in Complex Systems Implementation: The Case of Brazil''s Correspondent Banking System',
                'Revisiting Group-Based Technology Adoption as a Dynamic Process: The Role of Changing Attitude-Rationale Configurations',
                'Capturing the Complexity of Malleable IT Use: Adaptive Structuration Theory for Individuals',
                'A Temporally Situated Self-Agency Theory of Information Technology Reinvention'
                );'''
    df_entities= conn.execute(query).fetchdf()

In [15]:
print(df_entities)

        para_id              label                       ent_id  \
0         926_0              TOPIC             virtual teamwork   
1         926_0         TECHNOLOGY   IT supported collaboration   
2         926_1              STAGE                  prior study   
3         926_1              TOPIC             virtual teamwork   
4         926_1         TECHNOLOGY   IT supported collaboration   
...         ...                ...                          ...   
9039  12686_230  COLLECTION_METHOD       data collection method   
9040  12686_230  COLLECTION_METHOD        qualitative interview   
9041  10863_258             METRIC  no statistical significance   
9042  10863_258              TOPIC          technology adoption   
9043  10863_264  COLLECTION_METHOD            experimental task   

                 level_3  
0               IS topic  
1          IS technology  
2         research stage  
3               IS topic  
4          IS technology  
...                  ...  
9039  

In [16]:
df_entities = df_entities.groupby('para_id').agg({
    'ent_id': lambda x: ', '.join(sorted(set(x))),
    'level_3': lambda x: ', '.join(sorted(set(x))),
    'label': lambda x: ', '.join(sorted(set(x)))
}).reset_index()

In [17]:
print(df_entities)

       para_id                                             ent_id  \
0       1033_0                                      IS technology   
1       1033_1  IS technology, United States, individual level...   
2     1033_100                                   banking industry   
3     1033_101  banking industry, database system, individual ...   
4     1033_102  banking industry, database system, electronic ...   
...        ...                                                ...   
2296   9957_94                             individual participant   
2297   9957_95  business intelligence, database system, spread...   
2298   9957_96                                   individual level   
2299   9957_98  construct, self efficacy, technology adoption,...   
2300   9957_99                                technology adoption   

                                                level_3  \
0                                         IS technology   
1     IS technology, IS topic, geographic names, lev.

In [18]:
merged_df = merged_df.merge(df_entities, on=['para_id'])

In [19]:
print(merged_df)

      article_id    para_id  \
0            926      926_0   
1            926      926_1   
2            926      926_3   
3            926      926_4   
4            926      926_5   
...          ...        ...   
1824       12686  12686_211   
1825       12686  12686_212   
1826       12686  12686_213   
1827       12686  12686_215   
1828       12686  12686_216   

                                     last_section_title  \
0                                                  None   
1                                              Abstract   
2                                        Introduction 1   
3                                        Introduction 1   
4                                        Introduction 1   
...                                                 ...   
1824                                         Conclusion   
1825                                         Conclusion   
1826                                         Conclusion   
1827  Despite its limitations and ope

In [20]:
merged_df = merged_df.merge(df_article, on=['article_id'])

In [21]:
print(merged_df)

      article_id    para_id  \
0            926      926_0   
1            926      926_1   
2            926      926_3   
3            926      926_4   
4            926      926_5   
...          ...        ...   
1824       12686  12686_211   
1825       12686  12686_212   
1826       12686  12686_213   
1827       12686  12686_215   
1828       12686  12686_216   

                                     last_section_title  \
0                                                  None   
1                                              Abstract   
2                                        Introduction 1   
3                                        Introduction 1   
4                                        Introduction 1   
...                                                 ...   
1824                                         Conclusion   
1825                                         Conclusion   
1826                                         Conclusion   
1827  Despite its limitations and ope

In [30]:
null_counts = merged_df.isnull().sum()
print(null_counts)

article_id             0
para_id                0
last_section_title    22
paragraph              0
ent_id                 0
level_3                0
label                  0
authors                0
year                   0
title                  0
journal                0
abstract               0
keywords               0
citation_count         0
dtype: int64


In [31]:
merged_df = merged_df.fillna('No section information')

In [33]:
def concatenate_with_headers(df):
    concatenated_rows = []
    for index, row in df.iterrows():
        concatenated_row = " ".join([f"{col}: {row[col]}" for col in df.columns])
        concatenated_rows.append(concatenated_row)
    return concatenated_rows

In [34]:
documents = [
    Document(
        page_content=row['paragraph'], 
        metadata={
            'article_id': row['article_id'],
            'para_id': row['para_id'],
            'last_section_title': row['last_section_title'],
            'ent_id': row['ent_id'],
            'label': row['label'],
            'authors': row['authors'],
            'year': row['year'],
            'title': row['title'],
            'keywords': row['keywords'],
            'citation_count': row['citation_count']
        }
    )
    for _, row in merged_df.iterrows()
]

In [35]:
for doc in documents[:5]:
    print("Page Content:", doc.page_content)
    print("Metadata:", doc.metadata)

Page Content: NATURE AND NURTURE: THE IMPACT OF AUTOMATICITY AND THE STRUCTURATION OF COMMUNICATION ON VIRTUAL TEAM BEHAVIOR AND PERFORMANCE 
Metadata: {'article_id': 926, 'para_id': '926_0', 'last_section_title': 'No section information', 'ent_id': 'IT supported collaboration, virtual teamwork', 'label': 'TECHNOLOGY, TOPIC', 'authors': 'Bartelt, Valerie L.; Dennis, Alan R.', 'year': 2014, 'title': 'Nature and Nurture: The Impact of Automaticity and the Structuration of Communication on Virtual Team Behavior and Performance', 'keywords': 'Collaboration, virtual teams, genre rules, structuration theory, IM, discussion board, time pressure', 'citation_count': 4}
Page Content: Much prior research on virtual teams has examined the impact of the features and capabilities of different communication tools (the nature of communication) on team performance. In this paper, we examine how the social structures (i.e., genre rules) that emerge around different communication tools (the nurture of co

In [27]:
persist_directory = '../RAG_single_one_vector_database/all_info_chroma_db_MISQ'

In [36]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

In [37]:
vectordb_all = Chroma.from_documents(documents=documents,
                                    embedding=embedding_model,
                                    persist_directory=persist_directory)